In [ ]:
'''
Code based upon https://github.com/mfaruqui/retrofitting 
Reference paper: https://arxiv.org/abs/1411.4166 by Faruqui et al

'''

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [ ]:
import os
os.chdir('./My Drive/NLP_Project_Programs/Retrofitting_ESA')
os.listdir()

['Graphs',
 'Datasets',
 'OtherFiles',
 'Evaluation_Benchmarks',
 'TOEFL Task.ipynb',
 'Wikipedia-RG65_10titles.xml',
 'Wikipedia-RG65_20titles.xml',
 'Wikipedia-MEN_5titles.xml',
 'Wikipedia-MEN_3titles.xml',
 'Eval_Dataset_String.ipynb',
 'Graph_Processing.ipynb',
 'Sentiment_Analysis.ipynb',
 'Word Similarity Tasks.ipynb',
 'WS_6titles.txt',
 'ws353_6titles_1perline.txt',
 'titles_extract.ipynb',
 'explicit_semantic_anlaysis+retrofitting.ipynb']

In [ ]:
import numpy as np
import pandas as pd
from copy import deepcopy

from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from gensim.models import TfidfModel
from itertools import combinations
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm.notebook import tqdm
from gensim.corpora import Dictionary
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models.phrases import Phrases, Phraser
from sklearn.preprocessing import normalize

import json
import os
# %matplotlib qt
import nltk
nltk.download('stopwords')
nltk.download('punkt')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Set of all stopwords.
stop_words = set(stopwords.words('english'))

# Finds if in a given list of words there is a Bi-gram. Returns word1_word2 is Bi-gram else returns the word itself.
# bigram_edu = Phraser.load("./Datasets/bigram_data_wiki15.pkl")
# Returns the word after stemming.
ps = PorterStemmer()

def clean_data(d):
    ''' 
    Function to clean sentences and return list of tokens (words).
    '''
    # Split sentence into word tokens.
    res = word_tokenize(d)
    # Convert all word tokens to lower case.
    res = [token.lower() for token in res]
    # Keep only those words that are alphanumeric.
    res = [token for token in res if (token.isalnum())]

    # Stemming and removing stop words.
    tokens = []
    for token in res:
        tmp = ps.stem(token)
        # print(tmp)
        if tmp not in stop_words:
            tokens.append(tmp)
    other = []
    # Add Bi-Grams also as tokens.
    # print('DEBUG',tokens)
    # for k in bigram_edu[tokens]:
    #     # If k is Bi-Gram:
    #     if k not in tokens:
    #         other.append(k)
    # # Attach all Bi-grams to Word Tokens.
    # tokens.extend(other)
    return tokens

In [ ]:
'''
pth = 'OtherFiles/word_strings/'
p_ws353 = pth + 'ws353_word_string.txt'
p_rg = pth + 'rg_word_string.txt'
p_men = pth + 'men_word_string.txt'
p_sa_tr = pth + 'SA-train_word_string.txt'
p_sa_ts = pth +'SA-test_word_string.txt'
p_sa_trts = pth + 'SA-traintest_word_string.txt'

pth_list = [p_ws353,p_rg,p_men,p_sa_tr,p_sa_ts,p_sa_trts]
word_eval_list = []
count = 0
for path in pth_list:
  count+=1
  #if count==2:break
  file = open(path,'r')
  for index,line in tqdm(enumerate(file)):
    print(type(line))
    word_data = clean_data(line)
    print(type(word_data))
    word_eval_list.append(word_data)
  file.close()  
'''

<class 'str'>
<class 'list'>



<class 'str'>
<class 'list'>



<class 'str'>
<class 'list'>



<class 'str'>
<class 'list'>



<class 'str'>
<class 'list'>



<class 'str'>
<class 'list'>



In [ ]:
'''
set_word_eval_list = [0]*6

print("length of clean-data lists\n")
for i, token_list in tqdm(enumerate(word_eval_list)):
  print(pth_list[i],":length:",len(token_list))
  set_word_eval_list[i]=set(token_list)
  print("unique list length:",len(set(token_list)))

ws353_word_set = set_word_eval_list[0]
rg_word_set = set_word_eval_list[1]
men_word_set = set_word_eval_list[2]
satrain_word_set = set_word_eval_list[3]
satest_word_set = set_word_eval_list[4] 
satraintest_word_set = set_word_eval_list[5]
''' 

length of clean-data lists



OtherFiles/word_strings/ws353_word_string.txt :length: 706
unique list length: 434
OtherFiles/word_strings/rg_word_string.txt :length: 130
unique list length: 48
OtherFiles/word_strings/men_word_string.txt :length: 5998
unique list length: 740
OtherFiles/word_strings/SA-train_word_string.txt :length: 62828
unique list length: 9347
OtherFiles/word_strings/SA-test_word_string.txt :length: 16407
unique list length: 4902
OtherFiles/word_strings/SA-traintest_word_string.txt :length: 79235
unique list length: 10305



In [ ]:
'''
#Read the wiki article relations as a dictionary 
def read_lexicon(filename):
  lexicon = {}
  for line in open(filename, 'r',encoding = "utf8"):
    words = line.strip().split("--")
    lexicon[words[0]] = [word for word in words[1:]]
  return lexicon

lexicon = read_lexicon(filename = "./Graphs/wiki_graph_2_men5.txt")
graph_articles_set = []
for key, value in lexicon.items():
  graph_articles_set.append(key)
  for val in value:
    graph_articles_set.append(val)
graph_articles_set = set(graph_articles_set)
'''

In [ ]:
##For getting temp directory do the following:
#1. The wikipedia downloaded is given as input to "python xml_parse.py wikifile" 
##from https://github.com/bjarkemoensted/Wiki-ESA. This cleans wikidata and outputs temp directory with cleaned text
%%time
import random
random.seed(10)

# Store list of concepts description as list of tokens.
dataset = []
dataset_orig = []
# Store index-concept mapping as dictionary.
id2concept = {}
count = 0
# Find all text files within temp folder.
for filename in tqdm(os.listdir('./Datasets/ws6/temp/')):
    if filename.endswith(".raw"):
        # path to the file.
        # print('-'*25, filename)
        file = os.path.join('./Datasets/ws6/temp/', filename)
        # Open the file.
        with open(file) as json_file:
            # Load json.
            concepts = json.load(json_file)
            # print(concepts['Mercury'])
            # break
            # For each concept.
            for concept in concepts.keys():
                if concepts[concept]['links_out']:
                  orig_data = concepts[concept]['text']
                  len_document = len(concepts[concept]['text'].split(" "))
                  # if len_document>200 and len_document<250:
                  if len_document > 0:
                  # if concept in graph_articles_set:
                  # if random.random()<0.2: # 10% of the docs
                    # Update dictionary with key as count(index) and value as the name of concept.
                    id2concept[count] = concept
                    count+=1
                    # Clean the data present in 'text' field for the concept in json.
                    data = clean_data(orig_data)
                    # Append the list of tokens(words).
                    # dataset_orig.append(orig_data)
                    
                    '''
                    #----------For using Term-doc matrix containing words from Evaln. task alone--------
                    only_words = []
                    # <dataset>_word_set is cleaned list of word-tokens from chosen evaluation benchmark dataset (RG,MEN, SA.....)
                    for token in data:
                      if token in satraintest_word_set: 
                        only_words.append(token)
                    dataset.append(only_words)
                    '''
                    dataset.append(data)
                    # print(concept, concepts[concept])
                    # if count>50: break
                    # break
    else:
        continue
    # break


CPU times: user 2min 5s, sys: 68.2 ms, total: 2min 6s
Wall time: 2min 9s


In [ ]:
id2word = Dictionary(dataset)
sum([1 for _ in id2word.iterkeys()]), len(dataset)

(183592, 2555)

In [ ]:
import pickle

with open('Datasets/id2concept_smaller_ws6.pickle', 'wb') as handle:
    pickle.dump(id2concept, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Datasets/id2concept_smaller_ws6.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(id2concept == b)

with open('Datasets/dataset_smaller_ws6.pickle', 'wb') as handle:
    pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Datasets/dataset_smaller_ws6.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(dataset == b)

True
True


In [ ]:
# LOAD AND START FROM HERE. (There are ~230 files to read and it takes ~15 minutes. So just load from here) 
import pickle

with open('Datasets/id2concept_smaller_ws6.pickle', 'rb') as handle:
    id2concept = pickle.load(handle)

with open('Datasets/dataset_smaller_ws6.pickle', 'rb') as handle:
    dataset = pickle.load(handle)

In [ ]:
'''temp_dataset = [['bus','car','speed','mapple','ball','bus'],['when','wheress','mapple','whyss','whose','whom0'],['ball','fall','wheress','whom',]]
id2word = Dictionary(temp_dataset)
print("-"*20,id2word)
corpus = [id2word.doc2bow(line) for line in temp_dataset]
model = TfidfModel(corpus)

print("->"*3,corpus[0])

print(dict(model[corpus[0]])) 
print(model[corpus[0]])

print(".."*10,id2word.doc2bow(['what','why','where','mapple','ball']))
for x in id2word.itervalues():
  print(x)
'''

In [ ]:
%%time
# maps each unique word to an id.
id2word = Dictionary(dataset)
# corpus is list of documents. For each document in dataset create a bag-of-words list with each element of form (word_id,count).
corpus = [id2word.doc2bow(line) for line in dataset]
# Create the Tf-Idf model.
model = TfidfModel(corpus)

CPU times: user 6.88 s, sys: 49.8 ms, total: 6.93 s
Wall time: 6.93 s


In [ ]:
from scipy.sparse import csc_matrix, lil_matrix, dok_matrix
import time

def pre_process_tfidf(model, corpus, id2word):
  n_words = sum([1 for _ in id2word.iterkeys()])
  n_concepts = len(id2concept)
  print(n_words, n_concepts)
  tfidf_matrix = np.zeros((n_words, n_concepts))
  # tfidf_matrix = dok_matrix((n_words, n_concepts), dtype=np.float32)
  start = time.time()
  for concept_id in range(n_concepts):
    tfidf_concept = model[corpus[concept_id]]
    
    for word_id, score in tfidf_concept:
      tfidf_matrix[word_id, concept_id] = score
  print('Took =>', time.time()-start,'s(seconds)')
  return tfidf_matrix
  
tfidf_matrix = pre_process_tfidf(model, corpus, id2word)
# tfidf_matrix_orig = deepcopy(tfidf_matrix)

183592 2555
Took => 7.354796886444092 s(seconds)


In [ ]:
n_words_per_doc = []
for col_index in range(tfidf_matrix.shape[1]):
  n_words_per_doc.append(np.sum(tfidf_matrix[:,col_index]>0))

print("words/doc:",np.mean(n_words_per_doc))

n_docs_per_word = []
for row_index in range(tfidf_matrix.shape[0]):
  n_docs_per_word.append(np.sum(tfidf_matrix[row_index,:]>0))

print("docs/word:",np.mean(n_docs_per_word))

words/doc: 596.6262230919765
docs/word: 8.303085101747353


In [ ]:
def generate_baseline_wordvec(tfidf_matrix, model, id2word):
  print(tfidf_matrix.shape)
  tfidf_matrix_normalize = normalize(tfidf_matrix, axis=0)
  word_vec = {}
  for word_id in tqdm(range(tfidf_matrix_normalize.shape[0])):
    word_vec[id2word[word_id]] = tfidf_matrix_normalize[word_id,:]
  with open('Datasets/ws6/word_vec_baseline.pickle', 'wb') as handle:
    pickle.dump(word_vec, handle, protocol=pickle.HIGHEST_PROTOCOL)
  return word_vec
  
_ = generate_baseline_wordvec(tfidf_matrix, model, id2word)

(183592, 2555)


In [ ]:
''' Read the PPDB word relations as a dictionary '''
def read_lexicon(filename):
  lexicon = {}
  for line in open(filename, 'r',encoding = "utf8"):
    words = line.strip().split("--")
    lexicon[words[0]] = [word for word in words[1:]]
  return lexicon

lexicon = read_lexicon(filename = "Graphs/wiki_graph_3_ws6.txt")

In [ ]:
%%time
def retrofit(tfidf_matrix, lexicon, concept2id):
  '''newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue'''

  cvVocab = set([concept for _, concept in id2concept.items()])
  loopVocab = cvVocab.intersection(set(lexicon.keys()))
  print(len(loopVocab), len(cvVocab), len(set(lexicon.keys())))
  print(tfidf_matrix.shape, len(lexicon), len(concept2id))
  alpha = 1.0
  # tfidf_matrix_orig_bool = tfidf_matrix_orig>0
  # loop through every node also in ontology (else just use data estimate)
  tfidf_matrix = normalize(tfidf_matrix, axis=0)
  for concept in tqdm(loopVocab):
    conceptNeighbours = set(lexicon[concept]).intersection(cvVocab)
    numNeighbours = len(conceptNeighbours)
    # alpha = 1.0/numNeighbours
    #no neighbours, pass - use data estimate
    if numNeighbours == 0:
      continue
    '''# the weight of the data estimate if the number of neighbours
    newVec = numNeighbours * wordVecs[word]
    # loop over neighbours and add to new vector (currently with weight 1)
    for ppWord in wordNeighbours:
      newVec += newWordVecs[ppWord]
    newWordVecs[word] = newVec/(2*numNeighbours)'''
    # start = time.time()
    concept_id = concept2id[concept]
    # newVec = tfidf_matrix[:, concept_id].toarray()
    newVec = tfidf_matrix[:, concept_id]
    # boolVec = newVec>0
    # print(np.sum(boolVec), boolVec)
    # break
    # print(sum(boolVec), np.where(newVec>0)[0].shape)
    newVec = alpha * numNeighbours * newVec
    for ppConcept in conceptNeighbours:
      ppConcept_id = concept2id[ppConcept]
      # newVec += tfidf_matrix[:, ppConcept_id].toarray()
      newVec += tfidf_matrix[:, ppConcept_id]
    newVec = newVec/((alpha+1)*numNeighbours)
    # print(boolVec, newVec)
    # for row_index, value in enumerate(newVec):
    #   # if boolVec[row_index][0]:
    #   if boolVec[row_index]:
    #     # tfidf_matrix[row_index, concept_id] = value[0]
    #     tfidf_matrix[row_index, concept_id] = value
    tfidf_matrix[:, concept_id] = newVec
    # print('Took =>', time.time()-start,'s(seconds)')
    # break

  return tfidf_matrix

concept2id = {value:key for key, value in id2concept.items()}
# tfidf_matrix = tfidf_matrix.todok()
tfidf_matrix = retrofit(tfidf_matrix, lexicon, concept2id)

1767 2555 1768
(183592, 2555) 1768 2555



CPU times: user 43.7 s, sys: 271 ms, total: 44 s
Wall time: 43.8 s


In [ ]:
def generate_retrofit_wordvec(tfidf_matrix, model, id2word):
  print(tfidf_matrix.shape)
  word_vec = {}
  for word_id in tqdm(range(tfidf_matrix.shape[0])):
    word_vec[id2word[word_id]] = tfidf_matrix[word_id,:]
  with open('Datasets/ws6/word_vec_retrofit_1iter.pickle', 'wb') as handle:
    pickle.dump(word_vec, handle, protocol=pickle.HIGHEST_PROTOCOL)
  return word_vec
  
_ = generate_retrofit_wordvec(tfidf_matrix, model, id2word)

(183592, 2555)


In [ ]:
cvVocab = set([concept for _, concept in id2concept.items()])
loopVocab = cvVocab.intersection(set(lexicon.keys()))
sorted(list(cvVocab))[:5], sorted(list(set(lexicon.keys())))[:5]

In [ ]:
from sklearn.preprocessing import normalize
print(np.sum(tfidf_matrix[1,:]))

# print(np.sum(normalize(tfidf_matrix, axis=1)[1,:]))
x = normalize(tfidf_matrix, axis=1)
# x = tfidf_matrix/np.sum(tfidf_matrix, axis=1)
# x = np.divide(tfidf_matrix, np.sum(tfidf_matrix, axis=1)[:])
# np.sum(x[1,:])

5.280177131871051


In [ ]:
import math

math.sqrt((x[1,:]**2).sum())

1.0000000000000004

In [ ]:
boolVec = tfidf_matrix[:, 0]>0
boolVec[1] = False
(boolVec)*tfidf_matrix[:, 0], tfidf_matrix[:, 0]

(array([0.        , 0.00643216, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0.00400508, 0.00643216, 0.00904029, ..., 0.        , 0.        ,
        0.        ]))

In [ ]:
0*tfidf_matrix[:, 0]

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
 boolVec = tfidf_matrix[:, 0]>0
 boolVec*tfidf_matrix[:, 0], tfidf_matrix[:, 0]

(array([6.25793018e-05, 3.30841206e-04, 2.78590155e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 array([6.25793018e-05, 3.30841206e-04, 2.78590155e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))

In [ ]:
 boolVec = tfidf_matrix[:, 0]>0
 boolVec*tfidf_matrix[:, 0], tfidf_matrix[:, 0]

(array([0.00400508, 0.00643216, 0.00904029, ..., 0.        , 0.        ,
        0.        ]),
 array([0.00400508, 0.00643216, 0.00904029, ..., 0.        , 0.        ,
        0.        ]))

In [ ]:
tfidf_matrix[0,:].shape

(2521,)

In [ ]:
  with open('Datasets/tfidf_matrix.pickle', 'wb') as handle:
    pickle.dump(tfidf_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import numpy as np
np.save('Datasets/tfidf_matrix_npy', tfidf_matrix)

# Different versions of Retrofitting

In [ ]:
def retrofit(tfidf_matrix, lexicon, concept2id, numIters=1):
  '''newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue'''

  cvVocab = set([concept for _, concept in id2concept.items()])
  loopVocab = cvVocab.intersection(set(lexicon.keys()))
  print(len(loopVocab), len(cvVocab), len(set(lexicon.keys())))
  print(tfidf_matrix.shape, len(lexicon), len(concept2id))
  alpha = 0.0
  tfidf_matrix_orig_bool = tfidf_matrix_orig>0
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    
    for concept in tqdm(loopVocab):
      conceptNeighbours = set(lexicon[concept]).intersection(cvVocab)
      numNeighbours = len(conceptNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue


      '''# the weight of the data estimate if the number of neighbours
      newVec = numNeighbours * wordVecs[word]
      # loop over neighbours and add to new vector (currently with weight 1)
      for ppWord in wordNeighbours:
        newVec += newWordVecs[ppWord]
      newWordVecs[word] = newVec/(2*numNeighbours)'''
      # start = time.time()
      concept_id = concept2id[concept]
      # newVec = tfidf_matrix[:, concept_id].toarray()
      newVec = tfidf_matrix[:, concept_id]
      newVec = alpha * numNeighbours * newVec
      update_word_bool = tfidf_matrix_orig_bool[:,concept_id]
      intersec_words_bool = np.array([True for _ in range(tfidf_matrix.shape[0])])
      for ppConcept in conceptNeighbours:
        ppConcept_id = concept2id[ppConcept]
        intersec_words_bool = intersec_words_bool & tfidf_matrix_orig_bool[:,ppConcept_id]
        # newVec += tfidf_matrix[:, ppConcept_id].toarray()
        newVec += tfidf_matrix[:, ppConcept_id]
      # print(np.sum(intersec_words_bool))
      # break
      newVec = newVec/((alpha+1)*numNeighbours)
      # print(boolVec, newVec)
      # for row_index, value in enumerate(newVec):
        # # if boolVec[row_index][0]:
        # if boolVec[row_index]:
        #   # tfidf_matrix[row_index, concept_id] = value[0]
        #   tfidf_matrix[row_index, concept_id] = value
      tfidf_matrix[:, concept_id] = (intersec_words_bool & ~update_word_bool)*newVec + tfidf_matrix_orig[:,concept_id]
      # print('Took =>', time.time()-start,'s(seconds)')
      # break

  return tfidf_matrix

concept2id = {value:key for key, value in id2concept.items()}
# tfidf_matrix = tfidf_matrix.todok()
tfidf_matrix = retrofit(tfidf_matrix, lexicon, concept2id, numIters=1)

In [ ]:
def retrofit(tfidf_matrix, lexicon, concept2id, numIters=1):
  '''newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue'''

  cvVocab = set([concept for _, concept in id2concept.items()])
  loopVocab = cvVocab.intersection(set(lexicon.keys()))
  print(len(loopVocab), len(cvVocab), len(set(lexicon.keys())))
  print(tfidf_matrix.shape, len(lexicon), len(concept2id))
  alpha = 0.0
  tfidf_matrix_orig_bool = tfidf_matrix_orig>0
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for concept in tqdm(loopVocab):
      conceptNeighbours = set(lexicon[concept]).intersection(cvVocab)
      numNeighbours = len(conceptNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue


      '''# the weight of the data estimate if the number of neighbours
      newVec = numNeighbours * wordVecs[word]
      # loop over neighbours and add to new vector (currently with weight 1)
      for ppWord in wordNeighbours:
        newVec += newWordVecs[ppWord]
      newWordVecs[word] = newVec/(2*numNeighbours)'''
      # start = time.time()
      concept_id = concept2id[concept]
      # newVec = tfidf_matrix[:, concept_id].toarray()
      newVec = tfidf_matrix[:, concept_id]
      newVec = alpha * numNeighbours * newVec
      update_word_bool = tfidf_matrix_orig_bool[:,concept_id]
      union_words_bool = np.array([False for _ in range(tfidf_matrix.shape[0])])
      for ppConcept in conceptNeighbours:
        ppConcept_id = concept2id[ppConcept]
        union_words_bool = union_words_bool | tfidf_matrix_orig_bool[:,ppConcept_id]
        # newVec += tfidf_matrix[:, ppConcept_id].toarray()
        newVec += tfidf_matrix[:, ppConcept_id]
      # print(np.sum(union_words_bool))
      # break
      newVec = newVec/((alpha+1)*numNeighbours)
      # print(boolVec, newVec)
      # for row_index, value in enumerate(newVec):
        # # if boolVec[row_index][0]:
        # if boolVec[row_index]:
        #   # tfidf_matrix[row_index, concept_id] = value[0]
        #   tfidf_matrix[row_index, concept_id] = value
      tfidf_matrix[:, concept_id] = (union_words_bool & ~update_word_bool)*newVec + tfidf_matrix_orig[:,concept_id]
      # print('Took =>', time.time()-start,'s(seconds)')
      # break

  return tfidf_matrix

concept2id = {value:key for key, value in id2concept.items()}
# tfidf_matrix = tfidf_matrix.todok()
tfidf_matrix = retrofit(tfidf_matrix, lexicon, concept2id, numIters=1)

In [ ]:
def retrofit(tfidf_matrix, lexicon, concept2id, numIters=1):
  '''newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue'''

  cvVocab = set([concept for _, concept in id2concept.items()])
  loopVocab = cvVocab.intersection(set(lexicon.keys()))
  print(len(loopVocab), len(cvVocab), len(set(lexicon.keys())))
  print(tfidf_matrix.shape, len(lexicon), len(concept2id))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for concept in tqdm(loopVocab):
      conceptNeighbours = set(lexicon[concept]).intersection(cvVocab)
      numNeighbours = len(conceptNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue


      '''# the weight of the data estimate if the number of neighbours
      newVec = numNeighbours * wordVecs[word]
      # loop over neighbours and add to new vector (currently with weight 1)
      for ppWord in wordNeighbours:
        newVec += newWordVecs[ppWord]
      newWordVecs[word] = newVec/(2*numNeighbours)'''
      # start = time.time()
      concept_id = concept2id[concept]
      # newVec = tfidf_matrix[:, concept_id].toarray()
      newVec = tfidf_matrix[:, concept_id]
      boolVec = newVec>0
      # print(np.sum(boolVec), boolVec)
      # break
      # print(sum(boolVec), np.where(newVec>0)[0].shape)
      newVec = numNeighbours * newVec
      for ppConcept in conceptNeighbours:
        ppConcept_id = concept2id[ppConcept]
        # newVec += tfidf_matrix[:, ppConcept_id].toarray()
        newVec += tfidf_matrix[:, ppConcept_id]
      newVec = newVec/(2*numNeighbours)
      # print(boolVec, newVec)
      for row_index, value in enumerate(newVec):
        # if boolVec[row_index][0]:
        if boolVec[row_index]:
          # tfidf_matrix[row_index, concept_id] = value[0]
          tfidf_matrix[row_index, concept_id] = value
      # tfidf_matrix[:, concept_id] = newVec
      # print('Took =>', time.time()-start,'s(seconds)')
      # break

  return tfidf_matrix

concept2id = {value:key for key, value in id2concept.items()}
# tfidf_matrix = tfidf_matrix.todok()
tfidf_matrix = retrofit(tfidf_matrix, lexicon, concept2id, numIters=5)

2255 2521 2261
(203415, 2521) 2261 2521


In [ ]:
def retrofit(tfidf_matrix, lexicon, concept2id, numIters=1):
  '''newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue'''

  cvVocab = set([concept for _, concept in id2concept.items()])
  loopVocab = cvVocab.intersection(set(lexicon.keys()))
  print(len(loopVocab), len(cvVocab), len(set(lexicon.keys())))
  print(tfidf_matrix.shape, len(lexicon), len(concept2id))
  alpha = 1.0
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for concept in tqdm(loopVocab):
      conceptNeighbours = set(lexicon[concept]).intersection(cvVocab)
      numNeighbours = len(conceptNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue


      '''# the weight of the data estimate if the number of neighbours
      newVec = numNeighbours * wordVecs[word]
      # loop over neighbours and add to new vector (currently with weight 1)
      for ppWord in wordNeighbours:
        newVec += newWordVecs[ppWord]
      newWordVecs[word] = newVec/(2*numNeighbours)'''
      # start = time.time()
      concept_id = concept2id[concept]
      # newVec = tfidf_matrix[:, concept_id].toarray()
      newVec = tfidf_matrix[:, concept_id]
      # boolVec = newVec>0
      # print(np.sum(boolVec), boolVec)
      # break
      # print(sum(boolVec), np.where(newVec>0)[0].shape)
      newVec = alpha * numNeighbours * newVec
      for ppConcept in conceptNeighbours:
        ppConcept_id = concept2id[ppConcept]
        # newVec += tfidf_matrix[:, ppConcept_id].toarray()
        newVec += tfidf_matrix[:, ppConcept_id]
      newVec = newVec/((alpha+1)*numNeighbours)
      # print(boolVec, newVec)
      # for row_index, value in enumerate(newVec):
      #   # if boolVec[row_index][0]:
      #   if boolVec[row_index]:
      #     # tfidf_matrix[row_index, concept_id] = value[0]
      #     tfidf_matrix[row_index, concept_id] = value
      tfidf_matrix[:, concept_id] = newVec
      # print('Took =>', time.time()-start,'s(seconds)')
      # break

  return tfidf_matrix

concept2id = {value:key for key, value in id2concept.items()}
# tfidf_matrix = tfidf_matrix.todok()
tfidf_matrix = retrofit(tfidf_matrix, lexicon, concept2id, numIters=5)

# Retrofitting with different versions of Sparse Matrix

In [ ]:
def retrofit(tfidf_matrix, lexicon, concept2id, numIters=1):
  '''newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue'''

  cvVocab = set([concept for _, concept in id2concept.items()])
  loopVocab = cvVocab.intersection(set(lexicon.keys()))
  print(len(loopVocab), len(cvVocab), len(set(lexicon.keys())))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for concept in loopVocab:
      conceptNeighbours = set(lexicon[concept]).intersection(cvVocab)
      numNeighbours = len(conceptNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue


      '''# the weight of the data estimate if the number of neighbours
      newVec = numNeighbours * wordVecs[word]
      # loop over neighbours and add to new vector (currently with weight 1)
      for ppWord in wordNeighbours:
        newVec += newWordVecs[ppWord]
      newWordVecs[word] = newVec/(2*numNeighbours)'''
      start = time.time()
      concept_id = concept2id[concept]
      newVec = numNeighbours * tfidf_matrix[:, concept_id].toarray()
      for ppConcept in conceptNeighbours:
        ppConcept_id = concept2id[ppConcept]
        newVec += tfidf_matrix[:, ppConcept_id].toarray()
      newVec = newVec/(2*numNeighbours)
      for row_index, value in enumerate(newVec):
        tfidf_matrix[row_index, concept_id] = value
      print('Took =>', time.time()-start,'s(seconds)')
      break

  return

concept2id = {value:key for key, value in id2concept.items()}
retrofit(tfidf_matrix, lexicon, concept2id)

175559 175870 175820
Took => 20.26138687133789 s(seconds)


In [ ]:
def retrofit(tfidf_matrix, lexicon, concept2id, numIters=1):
  '''newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue'''

  cvVocab = set([concept for _, concept in id2concept.items()])
  loopVocab = cvVocab.intersection(set(lexicon.keys()))
  print(len(loopVocab), len(cvVocab), len(set(lexicon.keys())))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for concept in loopVocab:
      conceptNeighbours = set(lexicon[concept]).intersection(cvVocab)
      numNeighbours = len(conceptNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue


      '''# the weight of the data estimate if the number of neighbours
      newVec = numNeighbours * wordVecs[word]
      # loop over neighbours and add to new vector (currently with weight 1)
      for ppWord in wordNeighbours:
        newVec += newWordVecs[ppWord]
      newWordVecs[word] = newVec/(2*numNeighbours)'''
      start = time.time()
      concept_id = concept2id[concept]
      newVec = numNeighbours * tfidf_matrix[:, concept_id].toarray()
      for ppConcept in conceptNeighbours:
        ppConcept_id = concept2id[ppConcept]
        newVec += tfidf_matrix[:, ppConcept_id].toarray()
      newVec = newVec/(2*numNeighbours)
      for row_index, value in enumerate(newVec):
        tfidf_matrix[row_index, concept_id] = value
      print('Took =>', time.time()-start,'s(seconds)')
      break

  return

concept2id = {value:key for key, value in id2concept.items()}
# tfidf_matrix = tfidf_matrix.todok()
retrofit(tfidf_matrix, lexicon, concept2id)

175559 175870 175820
Took => 9.453941822052002 s(seconds)


In [ ]:
def retrofit(tfidf_matrix, lexicon, concept2id, numIters=1):
  '''newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue'''

  cvVocab = set([concept for _, concept in id2concept.items()])
  loopVocab = cvVocab.intersection(set(lexicon.keys()))
  print(len(loopVocab), len(cvVocab), len(set(lexicon.keys())))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for concept in loopVocab:
      conceptNeighbours = set(lexicon[concept]).intersection(cvVocab)
      numNeighbours = len(conceptNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue


      '''# the weight of the data estimate if the number of neighbours
      newVec = numNeighbours * wordVecs[word]
      # loop over neighbours and add to new vector (currently with weight 1)
      for ppWord in wordNeighbours:
        newVec += newWordVecs[ppWord]
      newWordVecs[word] = newVec/(2*numNeighbours)'''
      start = time.time()
      concept_id = concept2id[concept]
      newVec = numNeighbours * tfidf_matrix[:, concept_id].toarray()
      for ppConcept in conceptNeighbours:
        ppConcept_id = concept2id[ppConcept]
        newVec += tfidf_matrix[:, ppConcept_id].toarray()
      newVec = newVec/(2*numNeighbours)
      for row_index, value in enumerate(newVec):
        tfidf_matrix[row_index, concept_id] = value
      print('Took =>', time.time()-start,'s(seconds)')
      break

  return

concept2id = {value:key for key, value in id2concept.items()}
tfidf_matrix = tfidf_matrix.tocsc()
retrofit(tfidf_matrix, lexicon, concept2id)

175559 175870 175820


/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


KeyboardInterrupt: ignored

In [ ]:
def retrofit(tfidf_matrix, lexicon, concept2id, numIters=1):
  '''newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue'''

  cvVocab = set([concept for _, concept in id2concept.items()])
  loopVocab = cvVocab.intersection(set(lexicon.keys()))
  print(len(loopVocab), len(cvVocab), len(set(lexicon.keys())))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for concept in loopVocab:
      conceptNeighbours = set(lexicon[concept]).intersection(cvVocab)
      numNeighbours = len(conceptNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue


      '''# the weight of the data estimate if the number of neighbours
      newVec = numNeighbours * wordVecs[word]
      # loop over neighbours and add to new vector (currently with weight 1)
      for ppWord in wordNeighbours:
        newVec += newWordVecs[ppWord]
      newWordVecs[word] = newVec/(2*numNeighbours)'''
      start = time.time()
      concept_id = concept2id[concept]
      newVec = numNeighbours * tfidf_matrix[:, concept_id].toarray()
      for ppConcept in conceptNeighbours:
        ppConcept_id = concept2id[ppConcept]
        newVec += tfidf_matrix[:, ppConcept_id].toarray()
      newVec = newVec/(2*numNeighbours)
      for row_index, value in enumerate(newVec):
        tfidf_matrix[row_index, concept_id] = value
      print('Took =>', time.time()-start,'s(seconds)')
      break

  return

concept2id = {value:key for key, value in id2concept.items()}
tfidf_matrix = tfidf_matrix.tocsr()
retrofit(tfidf_matrix, lexicon, concept2id)

175559 175870 175820


/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


KeyboardInterrupt: ignored

In [ ]:
def retrofit(tfidf_matrix, lexicon, concept2id, numIters=1):
  '''newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue'''

  cvVocab = set([concept for _, concept in id2concept.items()])
  loopVocab = cvVocab.intersection(set(lexicon.keys()))
  print(len(loopVocab), len(cvVocab), len(set(lexicon.keys())))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for concept in loopVocab:
      conceptNeighbours = set(lexicon[concept]).intersection(cvVocab)
      numNeighbours = len(conceptNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue


      '''# the weight of the data estimate if the number of neighbours
      newVec = numNeighbours * wordVecs[word]
      # loop over neighbours and add to new vector (currently with weight 1)
      for ppWord in wordNeighbours:
        newVec += newWordVecs[ppWord]
      newWordVecs[word] = newVec/(2*numNeighbours)'''
      start = time.time()
      concept_id = concept2id[concept]
      newVec = numNeighbours * tfidf_matrix[:, concept_id].toarray()
      for ppConcept in conceptNeighbours:
        ppConcept_id = concept2id[ppConcept]
        newVec += tfidf_matrix[:, ppConcept_id].toarray()
      newVec = newVec/(2*numNeighbours)
      for row_index, value in enumerate(newVec):
        tfidf_matrix[row_index, concept_id] = value
      print('Took =>', time.time()-start,'s(seconds)')
      break

  return

concept2id = {value:key for key, value in id2concept.items()}
tfidf_matrix = tfidf_matrix.tobsr()
retrofit(tfidf_matrix, lexicon, concept2id)

175559 175870 175820


NotImplementedError: ignored

# Inverted Indexing

In [ ]:
for key, value in id2concept.items():
  if value=="María del Carmen Herrera Gómez":
    print('Gotcha,', key)

Gotcha, 101760


In [ ]:
# Dictionary with key as word_id and value as list of tuples of form (document_number, tf-idf score of word with word_id)
inverted_index = {}
# tfidf_matrix = tfidf_matrix.tocsc()

# Iterate over each document in corpus. (document is list of tuples of form (word_id,count))
for i in tqdm(range(len(corpus))):
    # Create dict with key as word_id and value as tf-idf score.
    '''tmp = dict(model[corpus[i]])'''
    # tmp = {word_id: score[0] for word_id, score in enumerate(tfidf_matrix[:,i].toarray()) if score[0]>0}
    tmp = {word_id: score for word_id, score in enumerate(tfidf_matrix[:,i]) if score>0}

    # For each word_id.
    for key in tmp.keys():
      
        # tf-idf score
        p = tmp[key]
# the mean is found to be 0.052 standard deviation is 0.056
#         if p < 0.069 + 0.066:
#             continue
        # Find the word for word_id.
        word = id2word[key]
        # If word is Bi-Gram.
        # if len(word.split('_')) == 2:
        #     # ???
        #     if p<0.02 + 0.028:
        #         continue
        # else:
        #     # ???
        #     if p < 0.02 + 0.036:
        #         continue
        # inverted_index dictionary with key as word_id and value as list. Add tuple (document_number,tf-idf score of word with word_id) to this list.
        try:
            r = inverted_index[key]
            inverted_index[key].append((i,p))
        except:
            inverted_index[key] = []
            inverted_index[key].append((i,p))

In [ ]:
# Find list of all uni-Grams and corresponding Tf-idf in each document. Goal is to find mean and standard deviation of tf-idf scores.
# bigrams = []
unigrams = []
for i in tqdm(range(len(corpus))):
    tmp = dict(model[corpus[i]])
    for key in tmp.keys():
        p = tmp[key]
        word = id2word[key]
        unigrams.append((word,p))
unigrams = np.array(unigrams)
# bigrams = np.array(bigrams)
print(np.shape(unigrams))
arr = (unigrams.T[1]).astype('float64')
np.mean(arr) # ???
np.sqrt(np.var(arr)) # ???


(3078898, 2)


ValueError: ignored

In [ ]:
# Find list of all Bi-Grams and corresponding Tf-idf in each document. Goal is to find mean and standard deviation of tf-idf scores.
bigrams = []
# unigrams = []
for i in tqdm(range(len(corpus))):
    tmp = dict(model[corpus[i]])
    for key in tmp.keys():
        p = tmp[key]
        word = id2word[key]
        if len(word.split('_')) == 2:
            bigrams.append((word,p))
# unigrams = np.array(unigrams)
bigrams = np.array(bigrams)
arr = (bigrams.T[1]).astype('float64')
np.mean(arr) # 0.0229327151184278
np.sqrt(np.var(arr)) # 0.028252774290045416

In [ ]:
len(np.where(arr>=0.02 + 0.032)[0])

64417

In [ ]:
x = (np.where(arr >= 0.2)[0])
print(bigrams[x])

[['enough_money' '0.2533101166256896']
 ['insert_money' '0.22966032701686828']
 ['state_machine' '0.39142253853209036']
 ...
 ['punjab_technical' '0.33705081170805345']
 ['ranjit_singh' '0.21313837634237864']
 ['metal_ions' '0.20165527895185367']]


In [ ]:
plt.hist(val_dist,bins = 'auto')

In [ ]:
val_dist = np.array(val_dist)

In [ ]:
gauss_dist = (val_dist-np.mean(val_dist))/(np.sqrt(np.var(val_dist)))

In [ ]:
plt.hist(gauss_dist,bins = 'auto')

In [ ]:
np.mean(val_dist)

In [ ]:
np.sqrt(np.var(val_dist))

In [ ]:
str('neural_network').split('_')

['neural', 'network']

In [ ]:
# Print all concepts with the word 'part_geometry' and corresponding tf-idf score.
word = 'part_geometry'
index = id2word.doc2bow([word])[0][0]
tmp = dict(inverted_index[index])
tmp = {k: v for k, v in sorted(tmp.items(), key=lambda item: item[1] ,reverse = True)}
for key in list(tmp.keys()):
    print(id2concept[key],tmp[key])

AutoForm 0.058959956138159825


In [ ]:
# model.save('../models/phrase/wiki_2005/model')
# id2word.save('../models/phrase/wiki_2005/id2word')
# np.save('../models/phrase/wiki_2005/id2concept.npy', id2concept) 
# np.save('../models/phrase/wiki_2005/inverted_index.npy', inverted_index) 

In [ ]:
##start from here
# id2concept = np.load('../models/phrase/wiki_2005/id2concept.npy', allow_pickle = "True").item()
# inverted_index = np.load('../models/phrase/wiki_2005/inverted_index.npy', allow_pickle = "True").item()
# model = TfidfModel.load('../models/phrase/wiki_2005/model')
# id2word = TfidfModel.load('../models/phrase/wiki_2005/id2word')

In [ ]:
df = pd.read_csv('./Datasets/cleaned_data.csv')
stop_words = set(stopwords.words('english'))

In [ ]:
# Same function as earlier.
def clean_data(d):
    res = word_tokenize(d)
    res = [token.lower() for token in res]
    res = [token for token in res if (token.isalnum()) ]
    tokens = []
    for token in res:
        tmp = ps.stem(token)
        if tmp not in stop_words:
            tokens.append(token)
    # other = []
    # for k in bigram_edu[tokens]:
    #     if k not in tokens:
    #         other.append(k)
    # tokens.extend(other)
    return tokens

# Store each course as 'decription... content... coursename' string.
data = []
for index,row in df.iterrows():
    data.append(row['description']+' '+row['content'] +' ' +row['coursename'])
    
# Clean the data for each course.
dataset = []
for d in data:
    d = clean_data(d)
    dataset.append(d)

In [ ]:
df.head()

In [ ]:
# Dictionary with key as course index (e.g: 1,2,3,...) and value as course number (e.g: AS2040 - Flight Dynamics I)
id2course = {}
# Dictionary with key as course number (e.g: AS2040 - Flight Dynamics I) and value as course index (e.g: 1,2,3,...)
course2id = {}
for index,row in df.iterrows():
    id2course[index] = row['course_index']
    course2id[row['course_index']] = index

In [ ]:
# Maps each unique word(value) to an id(key).
id2word_ = Dictionary(dataset)
# Maps each id(value) to word(key).
word2id_ = {}
for i in range(len(id2word_)):
    word2id_[id2word_[i]] = i
# corpus is list of documents. For each document in dataset create a bag-of-words list with each element of form (word_id,count).
corpus_ = [id2word_.doc2bow(line) for line in dataset]
# Create the Tf-Idf model.
model_ = TfidfModel(corpus_)

In [ ]:
# Function to map list of tuples of form (word_id from course corpus,tf-idf score) to list of tuples of form (word_id from wiki corpus,tf-idf score)
def transfrom_to(tmp):
    res = []
    for k in tmp:
        word = id2word_[k[0]]
        r = id2word.doc2bow([word])
        if len(r)>0:
            # Index/word_id from wikipedia corpus.
            index = r[0][0]
            # Append (index, tf-idf score)
            res.append((index,k[1]))
    return res

# Function to return list of tuples of form (document/concept number,tf-idf of word in course description x(times) tf-idf of word in concept).
def get_concept(tmp):
    # word_id in wiki corpus.
    index = tmp[0]
    # tf-idf score of word.
    val = tmp[1]
    try:
        # inverted_index: Dictionary with key as word_id and value as list of tuples of form (document_number, tf-idf score of word with word_id)
        # tmp1 : list of tuples of form (document_number, tf-idf score of word with word_id (here, index))
        tmp1 = inverted_index[index]
        res = []
        for k in tmp1:
            # k[0]: document_number/concept_number
            # k[1]: tf-idf score of word with word_id in wiki corpus.
            res.append((k[0], k[1]*val))
        return res
    except:
        return []

# Function that returns Dictionary with key as concept_number and value as sum of tf-idf_word_coursedescription x(times) tf-idf_word_concepts
def merged_concept(tmp):
    # list of tuples of form (word_id from wiki corpus,tf-idf score)
    tmp = transfrom_to(tmp)
    # res would be list of lists (one inner list for each word). 
    # Inner list contains tuples of form (document/concept number,tf-idf of word in course description x(times) tf-idf of word in concept).
    res = []
    # For word in course description.
    for k in tmp:
        r = get_concept(k)
        if len(r)>0:
            res.append(get_concept(k))
    ans = {}
    for k in res:
        # p: tuples of form (document/concept number,tf-idf of word in course description x (times) tf-idf of word in concept)
        # p[0]: concept number
        # p[1]: tf-idf of word in course description x(times) tf-idf of word in concept
        for p in k:
            try:
                t = ans[p[0]]
                ans[p[0]]+=p[1]
            except:
                ans[p[0]]=p[1]
    return ans

def tfidf_sim(a,b):
    i = course2id[a]
    j = course2id[b]
    # model_[corpus_[i/j]]: list of tuples of form (word_id from course corpus,tf-idf score) for i/j index document.
    # x: Dictionary with key as concept_number and value as sum of tf-idf_word_coursedescription x(times) tf-idf_word_concepts
    x = merged_concept(model_[corpus_[i]])
    y = merged_concept(model_[corpus_[j]])
    # Take intersection of concepts for x & y.
    tmp = list(set(x.keys()).intersection(set(y.keys())))
    # dot_product of x and y.
    val = 0
    for k in tmp:
        val+=(x[k]*y[k])
    den1 = 0
    den2 = 0
    # norm of x.
    for k in x.keys():
        den1+=(x[k]**2)
    # norm of y.
    for k in y.keys():
        den2+=(y[k]**2)
    # norm_x*norm_y
    den = np.sqrt(den1)*np.sqrt(den2)
    return val/den # Cosine Similarity

In [ ]:
# Probably for DEBUGGING...
a = 'course provides Concepts of reaction rates. Influence of concentration and temperature. Mixing in chemical reactors. Deterministic and stochastic modelling of reactor systems. Analysis of homogeneous and heterogeneous reactor theory with emphasis on concepts, mathematical formulation and solution for isothermal, adiabatic, non-isothermal non-adiabatic and programmed reactors. Product distribution, stability and optimization in chemical reactors.'
a = clean_data(a)
print(a)
tmp1 = id2word_.doc2bow(a)
print(model_[tmp1])
y = {k: v for k, v in sorted(merged_concept(tmp1).items(), key=lambda item: item[1], reverse = True)}
tmp = list(y.keys())
for k in tmp[0:5]:
    print(id2concept[k], y[k])

['course', 'provides', 'concepts', 'reaction', 'rates', 'influence', 'concentration', 'temperature', 'mixing', 'chemical', 'reactors', 'deterministic', 'stochastic', 'modelling', 'reactor', 'systems', 'analysis', 'homogeneous', 'heterogeneous', 'reactor', 'theory', 'emphasis', 'concepts', 'mathematical', 'formulation', 'solution', 'isothermal', 'adiabatic', 'programmed', 'reactors', 'product', 'distribution', 'stability', 'optimization', 'chemical', 'reactors', 'course_provides', 'reaction_rates', 'concentration_temperature', 'chemical_reactors', 'deterministic_stochastic', 'homogeneous_heterogeneous', 'mathematical_formulation', 'chemical_reactors']
[(12, 0.09275616530779447), (54, 0.0807363841874911), (59, 0.0866690579325808), (77, 0.032705591049422965), (116, 0.04714793278220359), (159, 0.03556012812960253), (217, 0.1497869509195821), (282, 0.15879781215143954), (350, 0.09587245607789228), (394, 0.07748446971128466), (587, 0.10552575145353826), (641, 0.029594657516467858), (734, 0.1

In [ ]:
# Probably for DEBUGGING...
word = 'course_provides'
index = id2word.doc2bow([word])[0][0]
print(model[id2word.doc2bow([word])])
tmp = dict(inverted_index[index])
tmp = {k: v for k, v in sorted(tmp.items(), key=lambda item: item[1] ,reverse = True)}
for key in list(tmp.keys()):
    print(id2concept[key],tmp[key])

[(122634, 1.0)]
Cisco certifications 0.030892460310081454
Marine Corps Communication Electronics School 0.024266222082513025
Communications School (United States Marine Corps) 0.023701940387800716
Master of Business Administration 0.00848536131793485


In [ ]:
cs_courses = np.load('./cs_courses.npy')
courses = np.load('./courses.npy')
rank_matrix = {}
# Python code to find 
# rank of elements 
def rankify_improved(A):  
    R = [0 for i in range(len(A))] 
    T = [(A[i], i) for i in range(len(A))]  
    T.sort(key=lambda x: x[0]) 
    (rank, n, i) = (1, 1, 0) 
    while i < len(A): 
        j = i  
        while j < len(A) - 1 and T[j][0] == T[j + 1][0]: 
            j += 1
        n = j - i + 1

        for j in range(n): 
            idx = T[i+j][1] 
            R[idx] = rank + (n - 1) * 0.5
        rank += n 
        i += n 
    return R  

for i in tqdm(range(len(cs_courses))):
    x = np.zeros(len(courses))
    for j in range(len(courses)):
        x[j]= (tfidf_sim(cs_courses[i],courses[j]))
    tmp = len(x) - np.array(rankify_improved(x))
    rank_matrix[i] = tmp

In [ ]:
#creating adjaceny list of all true relevant documents
adj = {}
with open('./edges.txt') as f:
    data = f.read().strip()
    data = eval(data)
for d in data:
    try:
        t = adj[d[0]]
        adj[d[0]].append(d[1])
    except:
        adj[d[0]] = []
        adj[d[0]].append(d[1])
Eval = []
for i in range(len(cs_courses)):
    val = 0
    for j in range(len(courses)):
        rel = 0
        if courses[j] in adj[cs_courses[i]]:
            rel = 1
        rank = rank_matrix[i][j]
        if(rel>0):
            val+= (rel/(np.log2( rank+1)))
    Eval.append(val)
#making normalized DCG for cs_courses
for i in range(len(Eval)):
    p = len(set(adj[cs_courses[i]]).intersection(set(courses)))
    val = 0
    for j in range(1,p+1):
        val += (1/(np.log2(j+1)))
    Eval[i]/=val
for i in range(len(cs_courses)):
    print(cs_courses[i],Eval[i])
print(np.mean(Eval))

CS1100 - Introduction to Programming 0.9512463365453543
CS1111 - Problem Solving using Computers 0.9273995247252625
CS2300 - Foundations of Computer Systems Design 0.8965508479224734
CS2310 - Foundations of Computer Systems Design Lab 0.6917585494894719
CS2700 - Programming and Data Structures 0.9287024045939805
CS2710 - Programming and Data Structures Lab 0.898350821706798
CS3100 - Paradigms of Programming 0.7829877839455167
CS3300 - Compiler Design 0.15186115333086322
CS3500 - Operating Systems 0.9828920819566879
CS3700 - Introduction to Database Systems 0.30478285340363576
CS5020 - Nonlinear optimisation: Theory and algorithms 0.8593688059794399
CS5691 - Pattern Recognition and Machine Learning 0.9318694681294111
CS5800 - Advanced Data Structures & Algorithms 0.8851433875425487
CS6014 - Computability and Complexity 0.7986469416219006
CS6015 - Linear Algebra and Random Processes 0.9021170980859685
CS6024 - Algorithmic Approaches to Computational Biology 0.40275295329000277
CS6030 - L

In [ ]:
#creating adjaceny list of all true relevant documents
adj = {}
with open('./Datasets/edges.txt') as f:
    data = f.read().strip()
    data = eval(data)
for d in data:
    try:
        t = adj[d[0]]
        adj[d[0]].append(d[1])
    except:
        adj[d[0]] = []
        adj[d[0]].append(d[1])
Eval = []
for i in range(len(cs_courses)):
    val = 0
    for j in range(len(courses)):
        rel = 0
        if courses[j] in adj[cs_courses[i]]:
            rel = 1
        rank = rank_matrix[i][j]
        if(rel>0):
            val+= (rel/(np.log2( rank+1)))
    Eval.append(val)
#making normalized DCG for cs_courses
for i in range(len(Eval)):
    p = len(set(adj[cs_courses[i]]).intersection(set(courses)))
    val = 0
    for j in range(1,p+1):
        val += (1/(np.log2(j+1)))
    Eval[i]/=val
for i in range(len(cs_courses)):
    print(cs_courses[i],Eval[i])
print(np.mean(Eval))

CS1100 - Introduction to Programming 0.9496882342166666
CS1111 - Problem Solving using Computers 0.9206931970139626
CS2300 - Foundations of Computer Systems Design 0.9001715370445234
CS2310 - Foundations of Computer Systems Design Lab 0.7033633977860031
CS2700 - Programming and Data Structures 0.9276164617947191
CS2710 - Programming and Data Structures Lab 0.8899047266703976
CS3100 - Paradigms of Programming 0.7685707357005617
CS3300 - Compiler Design 0.15019048322368797
CS3500 - Operating Systems 0.9047172294870751
CS3700 - Introduction to Database Systems 0.30478285340363576
CS5020 - Nonlinear optimisation: Theory and algorithms 0.8642002902821674
CS5691 - Pattern Recognition and Machine Learning 0.92220582424414
CS5800 - Advanced Data Structures & Algorithms 0.8811301180078074
CS6014 - Computability and Complexity 0.783783669291536
CS6015 - Linear Algebra and Random Processes 0.8932253221213741
CS6024 - Algorithmic Approaches to Computational Biology 0.40595036232755444
CS6030 - Log

In [ ]:
a = 'CS7016 - Topics In Deep Learning'
i = course2id[a]
x = merged_concept(model_[corpus_[i]])
y = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse = True)}
tmp = list(y.keys())
for k in tmp[0:20]:
    print(id2concept[k], y[k])
    
X = ['a' for j in range(20)]
ind = np.where(cs_courses == a)[0][0]
print(ind)
for index,k in enumerate(rank_matrix[ind]):
    if k<20:
        X[int(k)] = (id2course[index],k)
for k in X:
    print(k)

Deep learning 0.44257037762669066
Deep Learning Studio 0.27696621084785805
Artificial neural network 0.17560150969578958
Quantum neural network 0.15893451073982665
Recurrent neural network 0.14699050443657838
Neural Lab 0.13645957196360659
Neural Designer 0.11695990778792473
Neural engineering 0.11638124264837804
Challenger Deep 0.10871492864871551
Neural computation 0.1004093269566558
Adversarial machine learning 0.0918926883922573
Computational intelligence 0.08426501964331569
Deep belief network 0.08297624200258243
Deep sea fish 0.08172540348350787
DeepDream 0.08039485080489911
Wireless network 0.0787087347726274
Deep ocean water 0.07845746488517644
Learning 0.07545911108737599
Artificial intelligence 0.0732372204043199
Machine learning 0.06925176358803041
31
('CS7016 - Topics In Deep Learning', 0.0)
('CS7015 - Deep Learning', 1.0)
('ED6005 - Deep Learning for Medical Image Analysis', 2.0)
('EE4708 - Data Analytics Laboratory', 3.0)
('CS6777 - Optimization Methods for Computer Visio

In [ ]:
a = 'CS7016 - Topics In Deep Learning'
i = course2id[a]
x = merged_concept(model_[corpus_[i]])
y = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse = True)}
tmp = list(y.keys())
for k in tmp[0:20]:
    print(id2concept[k], y[k])
    
X = ['a' for j in range(20)]
ind = np.where(cs_courses == a)[0][0]
print(ind)
for index,k in enumerate(rank_matrix[ind]):
    if k<20:
        X[int(k)] = (id2course[index],k)
for k in X:
    print(k)

Deep learning 0.4534017903850849
Deep Learning Studio 0.29274101086405246
Artificial neural network 0.2291305518448815
Neural Designer 0.20470739305282928
Quantum neural network 0.20461925877290588
Recurrent neural network 0.19456965764344053
Neural Lab 0.18587353471741408
Neural engineering 0.15901090725837375
Challenger Deep 0.15895578871382654
Neural computation 0.1458890942862766
Deep ocean water 0.12181568066241026
Computational intelligence 0.12000997335777269
Machine learning 0.11707794460668505
Unsupervised learning 0.1125024243842067
Wireless network 0.10699225579464677
Deep sea fish 0.10542706884544253
DeepDream 0.10405741675850305
Q-learning 0.10372614874567813
Adversarial machine learning 0.10125241162638576
Learning rate 0.09825880390243119
31
('CS7016 - Topics In Deep Learning', 0.0)
('CS7015 - Deep Learning', 1.0)
('ED6005 - Deep Learning for Medical Image Analysis', 2.0)
('EE4708 - Data Analytics Laboratory', 3.0)
('CS6777 - Optimization Methods for Computer Vision Appl